In [3]:
import mysql.connector
import config

mydb = mysql.connector.connect(
	host="localhost",
	user=config.user,
	passwd=config.passwd,
	database="UrShop"
)
cursor=mydb.cursor(buffered=True)

In [2]:
import pandas as pd
data= pd.read_csv(r""+path)

NameError: name 'path' is not defined

checks for new user
number:- 10 digit mob number

return :

404 : if user not found
name of the user :- if found

In [90]:
def check_user(number):
    sql="SELECT name,user_id FROM user WHERE mob=\""+str(number)+"\""
    cursor.execute(sql)
    result=cursor.fetchone()
    if( result != None ):
        return result
    else:
        return "404"

In [91]:
check_user(8382930122)

('Aryan', 'UKA0001')

to check the types of client in an area

returns :
    GEN : General Store
    GRO : Grocery Store
    PHA : Pharmacy Store
    STA : Stationary Store

In [10]:
def client_type(pincode):
    sql="SELECT type FROM client WHERE pincode="+str(pincode)
    cursor.execute(sql)
    result=cursor.fetchall()
    return to_list(result)

client_name : returns name of the clients available in a pincode 

client_id : return id's of the clients available in the pincode

will be neede to execute at same time as the client id is only our reference 

In [61]:
def client_name(type, pincode):
    sql="SELECT name FROM client WHERE pincode="+str(pincode)+" AND type=\""+str(type)+"\""
    cursor.execute(sql)
    result=cursor.fetchall()
    return to_list(result)    

def client_id(type , pincode):
    sql="SELECT client_id FROM client WHERE pincode="+str(pincode)+" AND type=\""+str(type)+"\""
    cursor.execute(sql)
    result=cursor.fetchall()
    return to_list(result)    

will block the quantity in the inventory and also
will create a dump order until the payment is confirmed

In [86]:
def dump_order(client_id , items , quantity):
    stock=[]
    for s in get_sku(items):
        sql="SELECT in_stock,sold_stock FROM  "+str(clientid)+" WHERE sku="+str(s)
        cursor.execute(sql)
        result=cursor.fetchone()
        stock.append(result)

    updated_stock=[]
    for count,each in enumerate(stock):
        update=list(each)
        update[0]=update[0]-quantity[count]
        update[1]=update[1]+quantity[count]
        updated_stock.append(tuple(update))

    for count,sku in enumerate(get_sku(items)):
        sql="UPDATE "+str(clientid)+" SET in_stock=%s ,sold_stock=%s WHERE sku=%s"
        sql_list=list(updated_stock[count])
        sql_list.append(sku)
        val=tuple(sql_list)
        cursor.execute(sql , val)
        mydb.commit()   
    

will re-update the items if the payment fails

In [87]:
def re_update(clientid,items,quantity):
	stock=[]
	for s in get_sku(items):
		sql="SELECT in_stock FROM "+str(clientid)+" WHERE sku="+str(s)
		cursor.execute(sql)
		result=cursor.fetchone()
		stock.append(result)

	updated_stock=[]
	for count,each in enumerate(stock):
		update=list(each)
		update[0]=update[0]+quantity[count]
		updated_stock.append(tuple(update))

	for count,sku in enumerate(get_sku(items)):
		sql="UPDATE "+str(clientid)+" SET in_stock=%s WHERE sku=%s"
		sql_list=list(updated_stock[count])
		sql_list.append(sku)
		val=tuple(sql_list)
		cursor.execute(sql , val)
		mydb.commit()

In [76]:
def add_client(clientid):
	sql="CREATE TABLE "+str(clientid)+"(sku INT NOT NULL PRIMARY KEY,in_stock INT NOT NULL ,alarm INT NOT NULL ,sold_stock INT NOT NULL )"
	cursor.execute(sql)

insert items into inventory of specific client

PARAMETERS:-

client_id :- ID OF THE CLIENT
path_to_csv :- csv files with [sku,instock,alarm,sold_stock]

In [77]:
import pandas as pd
def add_in_bulk(client_id , path_to_csv):
    data= pd.read_csv(r""+path_to_csv)
    sql="INSERT INTO "+str(client_id)+"(sku, in_stock , alarm , sold_stock) VALUES(%s , %s , %s , %s)"
    for row in data.itertuples():
        val=(row.sku , row.in_stock , row.alarm , row.sold_stock)
        cursor.execute(sql , val)    
        mydb.commit()      

get sku of items :-

items :- accept list of items

In [82]:
def get_sku(items):
    sku=[]
    for item in items:
        sql="SELECT sku FROM items WHERE name=' "+item+"'"
        cursor.execute(sql)
        result=cursor.fetchone()
        sku.append(result[0])
    return sku

get_sku(['mushrooms' , 'rice' , 'butter'])

[21, 22, 26]

get stock of a certain client

client_id :- ID OF THE CLIENT
items:- LIST OF ITEMS

In [ ]:
def get_sku(items):
    sku=[]
    for item in items:
        sql="SELECT sku FROM items WHERE item_name=' "+item+"'"
        cursor.execute(sql)
        result=cursor.fetchone()
        sku.append(result[0])
    return sku

def get_stock(clientid,items):
	stock=[]
	for s in get_sku(items):
		sql="SELECT in_stock,sold_stock , alarm FROM "+str(clientid)+" WHERE sku="+str(s)
		cursor.execute(sql)
		result=cursor.fetchone()
		stock.append(result)
	return stock

inventory to be updated after each order :-

client_id :- ID OF THE CLIENT
items :- LIST OF THE ITEMS
quantity :- LIST OF THE QUANTITY OF ITEMS

In [78]:
def out_stock_update(clientid,items,quantity):
    stock=[]
    for s in get_sku(items):
        sql="SELECT in_stock,sold_stock FROM  "+str(clientid)+" WHERE sku="+str(s)
        cursor.execute(sql)
        result=cursor.fetchone()
        stock.append(result)

    updated_stock=[]
    for count,each in enumerate(stock):
        update=list(each)
        update[0]=update[0]-quantity[count]
        update[1]=update[1]+quantity[count]
        updated_stock.append(tuple(update))

    for count,sku in enumerate(get_sku(items)):
        sql="UPDATE "+str(clientid)+" SET in_stock=%s ,sold_stock=%s WHERE sku=%s"
        sql_list=list(updated_stock[count])
        sql_list.append(sku)
        val=tuple(sql_list)
        cursor.execute(sql , val)
        mydb.commit()

In [80]:
out_stock_update("CAG0001" , ['mushrooms' , 'rice' , 'butter'] , [1,1,1])

to be updated for specific items in client

client_id :- ID OF THE CLIENT
items :- LIST OF THE ITEMS
quantity :- LIST OF THE QUANTITY OF ITEMS

In [ ]:
def in_stock_update(clientid,items,quantity):
	stock=[]
	for s in get_sku(items):
		sql="SELECT in_stock FROM "+str(clientid)+" WHERE sku="+str(s)
		cursor.execute(sql)
		result=cursor.fetchone()
		stock.append(result)

	updated_stock=[]
	for count,each in enumerate(stock):
		update=list(each)
		update[0]=update[0]+quantity[count]
		updated_stock.append(tuple(update))

	for count,sku in enumerate(get_sku(items)):
		sql="UPDATE "+str(clientid)+" SET in_stock=%s WHERE sku=%s"
		sql_list=list(updated_stock[count])
		sql_list.append(sku)
		val=tuple(sql_list)
		cursor.execute(sql , val)
		mydb.commit()

to insert into master item list

In [ ]:
import pandas as pd
def add_in_bulk(path_to_csv):
    data= pd.read_csv(r""+path_to_csv)
    sql="INSERT INTO items(sku, name) VALUES(%s , %s)"
    for row in data.itertuples():
        val=(row.sku , row.item)
        cursor.execute(sql , val)
        mydb.commit()

to add new clients

In [ ]:
import pandas as pd
def add_in_bulk(path_to_csv):
    data= pd.read_csv(r""+path_to_csv)
    sql="INSERT INTO client(client_id,name,address,mob,alternate_mob,pincode,type,open_hour,close_hour) VALUES(%s , %s , %s , %s,%s , %s , %s , %s,%s)"
    for row in data.itertuples():
        val=(row.client_id , row.name , row.address , row.mob , row.alternate_mob ,row.pincode , row.type ,row.open_hour, row.close_hour)
        cursor.execute(sql , val)    
        mydb.commit()

to add new users

In [ ]:
import pandas as pd
def add_in_bulk(path_to_csv)
    data= pd.read_csv(r""+path_to_csv)
    sql="INSERT INTO user(user_id,name,address,mob,pincode) VALUES(%s , %s , %s , %s,%s)"
    for row in data.itertuples():
        val=(row.user_id , row.name , row.address , row.mob ,row.pincode)
        cursor.execute(sql , val)    
        mydb.commit()

to add new agent

In [ ]:
import pandas as pd
def add_in_bulk(path_to_csv)
    data= pd.read_csv(r""+path_to_csv)
    sql="INSERT INTO agent(agent_id,name,address,mob,alternate_mob,pincode) VALUES(%s , %s , %s , %s,%s,%s)"
    for row in data.itertuples():
        val=(row.agent_id , row.name , row.address , row.mob,row.alternate_mob ,row.pincode)
        cursor.execute(sql , val)    
        mydb.commit()

helper

In [6]:
def to_list(in_list):
    out_list=[]
    for type in in_list:
        out_list.append(type[0])
    return out_list        

to get tpye of clients in particular area

In [83]:
client_type(400052)

['GRO']

In [84]:
client_id("GEN",208011)

['CAG0001']

In [45]:
check_user(7904627765)

'Sathya'

In [69]:
def get_sku(items):
    sku=[]
    for item in items:
        sql="SELECT sku FROM items WHERE name=' "+item+"'"
        cursor.execute(sql)
        result=cursor.fetchone()
        sku.append(result[0])
    return sku

def get_stock(clientid,items):
    stock=[]
    for s in get_sku(items):
        sql="SELECT in_stock,sold_stock , alarm FROM "+str(clientid)+" WHERE sku="+str(s)
        cursor.execute(sql)
        result=cursor.fetchone()
        stock.append(result)
    return stock

In [71]:
get_stock("CAA0001",items=['mushrooms' , 'rice' , 'butter'])

[(257, 0, 20), (32, 0, 10), (11, 0, 3)]

In [113]:
import datetime
now = datetime.datetime.now()
time=now.strftime("%H:%M:%S")
date=now.strftime("%Y-%m-%d")